In [1]:
import numpy as np
from pandas import DataFrame, Series, read_csv

from sklearn.svm import OneClassSVM
from sklearn.metrics import f1_score, make_scorer
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

from bayes_opt import BayesianOptimization
from bayes_opt.logger import JSONLogger
from bayes_opt.event import Events

In [2]:
X_train = read_csv('../data/PAMAP2/x_train_data.csv')
X_valid = read_csv('../data/PAMAP2/x_val_data.csv')
y_train = read_csv('../data/PAMAP2/y_train_data.csv')
y_valid = read_csv('../data/PAMAP2/y_val_data.csv')

In [3]:
def filter_major_classes(y_classes:list, data: DataFrame, classes: list[int]):
    data['target'] = classes
    return (
        data[
            data['target'].isin(y_classes)
        ]
        .groupby('target')
        .apply(lambda x: x.sample(n=10000, random_state=42))
        .reset_index(drop=True)
    )

y_classes: list[int] = [1, 2, 3, 4] #, 16, 17]

data = filter_major_classes(y_classes, X_train, y_train)
X_train_balanced, y_train_balanced = data.drop(columns=['target']), data['target']

/tmp/ipykernel_62301/3846437325.py:8: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda x: x.sample(n=10000, random_state=42))


In [4]:
def score_function(model, X_train, y_true) -> float:
    return float(
        f1_score(
            np.ones(len(y_true)),
            model.predict(X_train),
            pos_label=1
        )
    )

def svm_target_function(nu:float, gamma:float) -> float:
    model = OneClassSVM(nu=nu, kernel='rbf', gamma=gamma).fit(X=X_train_balanced)
    # Calculate performance on the validation set and Convert the predictions:
    # 1 for normal and -1 for novelty. Returns the f1 score
    return score_function(model, X_train_balanced, y_train_balanced)

In [5]:
# Define bounds for nu and gamma
bounds = {
    'nu': (0.01, 0.5),
    'gamma': (0.001, 1)
}

# Run Bayesian Optimization
optimizer = BayesianOptimization(
    f=svm_target_function,
    pbounds=bounds,
    random_state=42
)
# optimizer.subscribe(Events.OPTIMIZATION_STEP, logger)
optimizer.maximize(init_points=5, n_iter=25)

print(f"Best result: {optimizer.max['params']}; f(x) = {optimizer.max['target']}.")

|   iter    |  target   |   gamma   |    nu     |
-------------------------------------------------
| 1         | 0.7236    | 0.3752    | 0.4759    |
| 2         | 0.6604    | 0.7323    | 0.3033    |
| 3         | 0.8324    | 0.1569    | 0.08644   |
| 4         | 0.7138    | 0.05903   | 0.4344    |
| 5         | 0.7672    | 0.6015    | 0.357     |
| 6         | 0.8644    | 0.02312   | 0.1172    |
| 7         | 0.8472    | 0.03517   | 0.01      |
| 8         | 0.8547    | 0.03606   | 0.125     |
| 9         | 0.739     | 0.3811    | 0.01      |
| 10        | 0.7069    | 1.0       | 0.01      |
| 11        | 0.8314    | 0.157     | 0.08637   |
| 12        | 0.771     | 0.1334    | 0.09865   |
| 13        | 0.896     | 0.0194    | 0.1345    |
| 14        | 0.9298    | 0.001154  | 0.1296    |
| 15        | 0.7225    | 0.5385    | 0.1689    |
| 16        | 0.9193    | 0.001     | 0.1494    |
| 17        | 0.9074    | 0.02176   | 0.1693    |
| 18        | 0.8967    | 0.001     | 0.1871    |


In [5]:
# Define the parameter grids
param_grid: dict[str, list[float]] = {
    'nu': [0.01, 0.05, 0.1, 0.2, 0.3, 0.5],
    'gamma': [0.001, 0.01, 0.1, 0.5, 1]
}

# Grid Search
grid_search: GridSearchCV[OneClassSVM] = GridSearchCV(
    estimator=OneClassSVM(kernel='rbf'),
    param_grid=param_grid,
    scoring=make_scorer(score_func=score_function),
    n_jobs=-1,
    verbose=3,
    # cv=5,
).fit(X_train)

Fitting 5 folds for each of 30 candidates, totalling 150 fits


KeyboardInterrupt: 

In [ ]:
# Random Search
random_search: RandomizedSearchCV = RandomizedSearchCV(
    estimator=OneClassSVM(kernel='rbf'),
    param_distributions=param_grid,
    n_iter=30,
    scoring=make_scorer(score_func=score_function),
    n_jobs=-1,
    # cv=5,
    verbose=3,
    random_state=42,
).fit(X_train)

In [ ]:
# Extract the best results
bayesian_best_params: dict[str, float] = optimizer.max['params']
grid_best_params: dict[str, float] = grid_search.best_params_
random_best_params: dict[str, float] = random_search.best_params_

print("Bayesian Optimization Best Params:", bayesian_best_params)
print("Grid Search Best Params:", grid_best_params)
print("Random Search Best Params:", random_best_params)

# You can further test these models on your test set and compare their performance